In [1]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

In [2]:
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.download('BTC-USD')
btc_df = btc_df.reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Open','High','Low','Adj Close', 'Volume', 'Unnamed: 0', "wallets", "address", "mined"])
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df = df.dropna()
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3
df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = df['price-meanavge']/(df['price'] + df['meanavge'])
bins = [-0.43, -0.1, 0, 0.1, 0.43]
group_names = ["Severely Oversold","Oversold", "Neutral","Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)
pricefrommean = df.meanvalue.iloc[-1].round(2)
currentzone = df.Valuation.iloc[-1]
delta1cycle = df.index[df['date']=='2015-09-15'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2cycle = df.index[df['date']=='2019-04-2'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averageunder = int((delta1cycle+delta2cycle)/2)
delta1fromp2p = df.index[df['date']=='2017-03-17'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2fromp2p = df.index[df['date']=='2020-11-29'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averagep2p = int((delta1fromp2p+delta2fromp2p)/2)
sincealltimehigh = df.index[-1] - df.index[df['price']==df.price.max()].tolist()[0]

[*********************100%***********************]  1 of 1 completed


In [3]:
import plotly.graph_objects as go
import plotly.io as pio

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = sincealltimehigh,
    mode = "gauge+number+delta",
    title = {'text': "Approximate Days to reach previous ATH"},
    delta = {'reference': averagep2p},
    gauge = {'axis': {'range': [None, averagep2p]},
    'bar': {'color': "orange"},}))
fig.show()
fig.write_html("../static/daysATH.html")

In [4]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1],
    mode = "gauge+number+delta",
    title = {'text': "Current Price and ATH Delta"},
    delta = {'reference': df.price[df['price']==df.price.max()].tolist()[0]},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/priceath.html")

In [5]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = sincealltimehigh,
    mode = "gauge+number+delta",
    title = {'text': "Approximate Days Price < Meanaverage"},
      delta = {'reference': averageunder},
    gauge = {'axis': {'range': [None, averageunder],},
    'bar': {'color': "orange"},}))


fig.show()
fig.write_html("../static/daysUnder.html")

In [6]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1] - df.meanavge.iloc[-1],
    mode = "gauge+number",
    title = {'text': "Current Price Meanaverage Delta"},
    delta = {'reference': 0},
    gauge = {'axis': {'range': [None, df.meanvalue[df['meanvalue']==df.meanvalue.min()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/price-mean.html")

In [7]:
df

,date,price,meanavge,meanvalue,status,price-meanavge,move%,Valuation
299,2011-07-21,13.600000,8.704611,4.895389,1,4.895389,0.219479,Overbought
300,2011-07-22,13.700000,8.762722,4.937278,1,4.937278,0.219799,Overbought
301,2011-07-23,13.700000,8.796167,4.903833,1,4.903833,0.217985,Overbought
302,2011-07-24,14.000000,8.801778,5.198222,1,5.198222,0.227974,Overbought
303,2011-07-25,14.100000,8.823000,5.277000,1,5.277000,0.230205,Overbought
...,...,...,...,...,...,...,...,...
4438,2022-12-17,16795.091797,21563.954649,-4768.862852,0,-4768.862852,-0.124322,Severely Oversold
4439,2022-12-18,16757.976562,21492.639639,-4734.663077,0,-4734.663077,-0.123780,Severely Oversold
4440,2022-12-19,16439.679688,21417.075424,-4977.395737,0,-4977.395737,-0.131480,Severely Oversold
4441,2022-12-20,16906.304688,21349.245423,-4442.940736,0,-4442.940736,-0.116138,Severely Oversold
